In [144]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score

In [102]:
data=pd.read_csv('C:/Users/sys/Downloads/competitive-data-science-predict-future-sales/sales_train_v2.csv')

In [103]:
shops=pd.read_csv('C:/Users/sys/Downloads/competitive-data-science-predict-future-sales/shops.csv')
items=pd.read_csv('C:/Users/sys/Downloads/competitive-data-science-predict-future-sales/items.csv')
item_catg=pd.read_csv('C:/Users/sys/Downloads/competitive-data-science-predict-future-sales/item_categories.csv')
test=pd.read_csv('C:/Users/sys/Downloads/competitive-data-science-predict-future-sales/test.csv.gz')

In [104]:
data.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
date              object
date_block_num    int64
shop_id           int64
item_id           int64
item_price        float64
item_cnt_day      float64
dtypes: float64(2), int64(3), object(1)
memory usage: 134.4+ MB


In [105]:
data.date #here date is of type object, we have to convert to datetime format

0          02.01.2013
1          03.01.2013
2          05.01.2013
3          06.01.2013
4          15.01.2013
              ...    
2935844    10.10.2015
2935845    09.10.2015
2935846    14.10.2015
2935847    22.10.2015
2935848    03.10.2015
Name: date, Length: 2935849, dtype: object

In [106]:
data['date']=pd.to_datetime(data['date'],format='%d.%m.%Y') #now our date is in datetime format

In [107]:
data.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-01-02,0,59,22154,999.00,1.0
1,2013-01-03,0,25,2552,899.00,1.0
2,2013-01-05,0,25,2552,899.00,-1.0
3,2013-01-06,0,25,2554,1709.05,1.0
4,2013-01-15,0,25,2555,1099.00,1.0


In [108]:
#to remove outliers 

data=data[(data.item_price>0)&(data.item_price<25000)]

In [109]:
data.shape

(2933475, 6)

In [93]:
data.groupby(['date_block_num','shop_id','item_id','item_price',pd.Grouper(key='date',freq='M')]).agg({'item_cnt_day':sum})
data.shape

(2933475, 6)

In [72]:
data

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-01-02,0,59,22154,999.00,1.0
1,2013-01-03,0,25,2552,899.00,1.0
2,2013-01-05,0,25,2552,899.00,-1.0
3,2013-01-06,0,25,2554,1709.05,1.0
4,2013-01-15,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,2015-10-10,33,25,7409,299.00,1.0
2935845,2015-10-09,33,25,7460,299.00,1.0
2935846,2015-10-14,33,25,7459,349.00,1.0
2935847,2015-10-22,33,25,7440,299.00,1.0


In [73]:
data['month']=data['date'].dt.month

In [74]:
data['year']= data['date'].dt.year

In [115]:
data.shape

(2933475, 6)

In [127]:
x=data.iloc[:,1:-1].values
y=data.iloc[:,5].values

In [128]:
print(x)

[[    0.    59. 22154.   999.]
 [    0.    25.  2552.   899.]
 [    0.    25.  2552.   899.]
 ...
 [   33.    25.  7459.   349.]
 [   33.    25.  7440.   299.]
 [   33.    25.  7460.   299.]]


In [129]:
print(y)

[ 1.  1. -1. ...  1.  1.  1.]


In [130]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=12)

In [131]:
gbreg=GradientBoostingRegressor(learning_rate=0.02)

In [132]:
gbreg.fit(x_train,y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.02, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [133]:
predictions=gbreg.predict(x_test)

In [134]:
predictions

array([1.24902712, 1.11295176, 1.3411366 , ..., 1.1825993 , 1.11973216,
       1.20725142])

In [135]:
gbreg.score(x_test,y_test)

0.044954844974742114

In [140]:
est1=LinearRegression()
est2=DecisionTreeRegressor()
est3=Lasso(alpha=0.5)

In [142]:
votreg=VotingRegressor(estimators=[
    ('Linear',est1),
    ('decision',est2),
    ('lasso',est3)
])

In [143]:
votreg.fit(x_train,y_train)

VotingRegressor(estimators=[('Linear',
                             LinearRegression(copy_X=True, fit_intercept=True,
                                              n_jobs=None, normalize=False)),
                            ('decision',
                             DecisionTreeRegressor(criterion='mse',
                                                   max_depth=None,
                                                   max_features=None,
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   presort=False,
                                          

In [145]:
votreg.score(x_test,y_test)

0.2332277350675801

In [147]:
predictions=votreg.predict(x_test)

In [148]:
r2_score(y_test,predictions)

0.2332277350675801